In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import math
dataV1= pd.read_csv(r"C:\Users\Rohit\Desktop\IEOR135Project\TrainingDatasetV2.csv",encoding = "latin1")
dataV1.head()

C:\Users\Rohit\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (12,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Birth Year,Gender Flag,Skillset1,Skillset1 Weight,Skillset2,Skillset2 Weight,City of profile,Country of profile,Education,...,Company,Company_Norm,Ticker,Exchange,Location,Industry,Layoff Date,Company Name,Employees Affected,laid_off
0,6b89d91d-7b71-37fe-bb4c-0084018d63c0,2001,0,Software Engineering,0.570285,Web Development,0.277136,San Ramon,United States,0,...,VMware,VMware,VMW,XNYS,"san francisco, ca",541512,3/12/2018,"VMware, Inc",159,0.0
1,6b89d91d-7b71-37fe-bb4c-0084018d63c0,2001,0,Software Engineering,0.570285,Web Development,0.277136,San Ramon,United States,0,...,VMware,VMware,VMW,XNYS,"san francisco, ca",541512,3/28/2016,"VMware, Inc",170,0.0
2,10bbd07e-09b0-3e33-9b6d-897b2d18ad43,1985,0,Software Engineering,0.664485,Web Development,0.236152,San Francisco,United States,5,...,Intel Corporation,Intel Corporation,INTC,XNAS,"portland, oregon area",334413,6/30/2018,Intel Corporation,65,0.0
3,10bbd07e-09b0-3e33-9b6d-897b2d18ad43,1985,0,Software Engineering,0.664485,Web Development,0.236152,San Francisco,United States,5,...,Intel Corporation,Intel Corporation,INTC,XNAS,"portland, oregon area",334413,2/24/2017,Intel Corporation,54,0.0
4,10bbd07e-09b0-3e33-9b6d-897b2d18ad43,1985,0,Software Engineering,0.664485,Web Development,0.236152,San Francisco,United States,5,...,Intel Corporation,Intel Corporation,INTC,XNAS,"portland, oregon area",334413,7/15/2015,Intel Corporation,165,0.0


In [2]:
not_laid_off = dataV1[dataV1['laid_off'] == 0]
laid_off = dataV1[dataV1['laid_off'] == 1]
print(len(not_laid_off)) # Too many observations classified as not laid off
print(len(laid_off))

np.random.seed(101)
drop_indices = np.random.choice(not_laid_off.index, 90000, replace=False)
not_laid_off = not_laid_off.drop(drop_indices)

dataV1 = pd.concat([not_laid_off, laid_off])
print('Training data size after dropping excessive observations: ', dataV1.shape)

98061
8040
Training data size after dropping excessive observations:  (16101, 29)


In [3]:

# use one-hot encoding to encode skillsets
encodeSkill1 = pd.get_dummies(dataV1['Skillset1'], prefix = 's1', prefix_sep='-')
encodeSkill2 = pd.get_dummies(dataV1['Skillset2'], prefix = 's2', prefix_sep='-') 
dataV1 = dataV1.drop(['Skillset1','Skillset2'], axis = 1)
dataV1 = dataV1.join(encodeSkill1) 
dataV1 = dataV1.join(encodeSkill2)
dataV1.head()

,ID,Birth Year,Gender Flag,Skillset1 Weight,Skillset2 Weight,City of profile,Country of profile,Education,Elite Institution,Start Date,...,s2-Product Management,s2-Public Policy,s2-Real Estate,s2-Retail and Fashion,s2-Sales,s2-Social Media and Communications,s2-Software Engineering,s2-Video and Film Production,s2-Web Design,s2-Web Development
29,070de6c5-fd69-313c-80ae-dcc7c9b6a7cd,1994,1,0.489101,0.227604,San Francisco,United States,4,1,2015/6/1,...,0,0,0,0,0,0,0,0,0,0
40,24c681a0-5154-3cfe-b9fc-dab08e2211fe,2001,0,0.816397,0.123156,San Francisco,United States,6,1,2007/1/1,...,1,0,0,0,0,0,0,0,0,0
50,7c33c06a-c6eb-3f97-9186-88bf82cc3112,2001,2,0.690564,0.112917,San Francisco,United States,0,0,1997/6/1,...,0,0,0,0,0,0,1,0,0,0
58,e790de02-4e1b-331e-8fef-af0aebc5479c,1969,2,0.478578,0.435329,San Francisco,United States,4,0,1989/1/1,...,0,0,0,0,1,0,0,0,0,0
78,3d61a61d-1f4f-3cb6-9712-d75e5930f06d,1995,2,0.238093,0.165229,San Francisco,United States,4,1,2014/5/1,...,0,0,0,0,0,0,0,0,0,0


In [4]:

dataV1 = dataV1.dropna(subset=['laid_off'])
dataV1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16101 entries, 29 to 106107
Data columns (total 91 columns):
ID                                         16101 non-null object
Birth Year                                 16101 non-null object
Gender Flag                                16101 non-null int64
Skillset1 Weight                           16101 non-null float64
Skillset2 Weight                           16101 non-null float64
City of profile                            14712 non-null object
Country of profile                         16101 non-null object
Education                                  16101 non-null int64
Elite Institution                          16101 non-null int64
Start Date                                 16101 non-null object
StartFlag                                  16101 non-null object
End Date                                   16101 non-null object
EndFlag                                    16101 non-null int64
CurrentEmployFlag                          161

In [5]:
dataV1['Length Group']=pd.cut(dataV1['Length'], bins=[0, 180, 360, 540,720,20000], 
                                             include_lowest=True, 
                                             labels=[1,2,3,4,5])


In [6]:
dataV1=dataV1.drop(['ID','Birth Year','Gender Flag','Skillset1 Weight','Skillset2 Weight','Company','ID',
       'City of profile', 'Country of profile','Elite Institution',
       'Start Date', 'StartFlag', 'End Date',
       'EndFlag', 'CurrentEmployFlag', 'Role', 'Dept', 'Company',
       'Company_Norm', 'Ticker', 'Exchange',
       'Industry','Layoff Date','Length'],axis=1)

In [7]:
dataV1=dataV1.dropna()

In [8]:
dataV1.columns

Index(['Education', 'Location', 'Company Name', 'Employees Affected',
       'laid_off', 's1-Administration', 's1-Business Development',
       's1-Data Analysis', 's1-Design', 's1-Digital Marketing',
       's1-Electrical Engineering', 's1-Energy, Oil, and Gas', 's1-Finance',
       's1-Healthcare', 's1-Hospitality', 's1-Human Resources',
       's1-IT Management and Support', 's1-Industrial Management',
       's1-Insurance', 's1-Legal', 's1-Logistics',
       's1-Manufacturing and Process Management', 's1-Military',
       's1-Mobile Telecommunications', 's1-Musical Production',
       's1-Non-Profit and Community', 's1-Operations Management',
       's1-Product Management', 's1-Public Policy', 's1-Real Estate',
       's1-Retail and Fashion', 's1-Sales',
       's1-Social Media and Communications', 's1-Software Engineering',
       's1-Video and Film Production', 's1-Web Design', 's1-Web Development',
       's2-Administration', 's2-Business Development', 's2-Data Analysis',
      

In [9]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
y = dataV1['laid_off']
X = dataV1.drop(['laid_off'],axis=1)



In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

C:\Users\Rohit\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
X.drop(['Employees Affected'])

KeyError: "['Employees Affected'] not found in axis"

In [24]:

logreg.score(X_test,y_test)

0.5838658146964856

In [11]:
a=np.asarray(dataV1['Length Group'].values.tolist())
a=a.astype(np.int32)
dataV1['Length Group']=pd.Series(a)

In [265]:
dataV1['Length Group']=pd.Series(a)

In [25]:
# First XGBoost classifier
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

# fit model no training data
#need to implement more rigorous hyperparameter tuning
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 61.18%
Accuracy: 61.18%


In [18]:
y_train

104204    1.0
105080    1.0
54568     0.0
8511      0.0
105785    1.0
44740     0.0
94438     0.0
103278    1.0
24122     0.0
102252    1.0
3710      0.0
74453     0.0
100267    1.0
9854      0.0
101534    1.0
101662    1.0
23545     0.0
104355    1.0
101171    1.0
106012    1.0
70307     0.0
100938    1.0
98136     1.0
3189      0.0
102954    1.0
100931    1.0
8122      0.0
101515    1.0
40        0.0
100507    1.0
         ... 
105618    1.0
102835    1.0
93982     0.0
29319     0.0
44310     0.0
27074     0.0
73235     0.0
103570    1.0
102774    1.0
769       0.0
75124     0.0
102039    1.0
99734     1.0
48484     0.0
98150     1.0
98898     1.0
105521    1.0
84479     0.0
38064     0.0
98079     1.0
96698     0.0
13754     0.0
65053     0.0
105929    1.0
98525     1.0
8421      0.0
97052     0.0
100717    1.0
18878     0.0
21528     0.0
Name: laid_off, Length: 8761, dtype: float64

In [261]:
dataV1['Length Group']

29       5.0
40       5.0
50       5.0
58       1.0
78       5.0
85       5.0
89       1.0
110      2.0
122      4.0
134      5.0
144      5.0
181      2.0
189      2.0
191      2.0
198      5.0
202      4.0
236      5.0
261      2.0
285      5.0
337      5.0
346      1.0
353      5.0
359      1.0
361      5.0
433      1.0
436      5.0
438      3.0
449      5.0
468      5.0
475      5.0
        ... 
15618    4.0
15635    5.0
15652    5.0
15654    5.0
15659    1.0
15660    3.0
15671    5.0
15691    5.0
15705    4.0
15717    1.0
15718    5.0
15744    5.0
15751    4.0
15758    4.0
15767    5.0
15774    3.0
15779    4.0
15810    5.0
15843    5.0
15863    5.0
15884    1.0
15889    5.0
15925    5.0
15943    5.0
15947    5.0
16006    3.0
16023    5.0
16050    5.0
16080    3.0
16089    3.0
Name: Length Group, Length: 1349, dtype: float64

In [201]:
dataV1.dropna(subset=['Length Group'])

dataV1['Length Group'].astype(np.int32)


ValueError: Cannot convert non-finite values (NA or inf) to integer